# Wildfires

## 1. Import Libraries

Import all the libraries we'll need for our analysis:


In [ ]:
import geopandas as gpd
import folium
import branca
from pathlib import Path

## 2. Load and Clean the Data

Load wildfire perimeter data from a GeoJSON file and prepare it for analysis.


In [ ]:
file_name = "WFIGS_Interagency_Perimeters_YTD.geojson"
wf_perim_path = Path.cwd() / file_name
wf_perims_raw = gpd.read_file(wf_perim_path)

# Display the raw data
wf_perims_raw.head()

### View All Columns

In [ ]:
wf_perims_raw.columns.tolist()

### Select and Clean Columns of Interest

In [ ]:
cols = [
    "attr_EstimatedCostToDate",  # Cost of firefighting efforts
    "attr_IncidentName",  # Name of the wildfire incident
    "poly_GISAcres",  # Size of the fire in acres
    "poly_IRWINID", # IRWIN ID
    "geometry",
]

# Create a clean dataset with only the columns we need
wf_perims = wf_perims_raw[cols].copy()

In [ ]:
# Fill missing values with zeros
wf_perims["attr_EstimatedCostToDate"] = wf_perims.attr_EstimatedCostToDate.fillna(value=0)
wf_perims["poly_GISAcres"] = wf_perims["poly_GISAcres"].fillna(value=0)

In [ ]:
# Filter fires below a certain size and cost
wf_perims = wf_perims[wf_perims["poly_GISAcres"] >= .01]
wf_perims = wf_perims[wf_perims["attr_EstimatedCostToDate"] >= 0]

# Display the first few rows of our cleaned dataset
wf_perims.head()

## 3. Basic Mapping With Folium

In [ ]:
# Create a base map
map1 = folium.Map(location=[37.406, -100.110], tiles="CartoDB Positron", zoom_start=5)

# Add wildfire perimeters
folium.GeoJson(
    wf_perims,
).add_to(map1)

# Display the map
map1

## 4. Styling With Branca

In [ ]:
# Create a color gradient for fire size
colormap_size = branca.colormap.LinearColormap(
    colors=[
        "green",
        "yellow",
        "orange",
        "red",
    ],  # Color progression from small to large
    vmin=wf_perims["poly_GISAcres"].quantile(0.0),  # Minimum value
    vmax=wf_perims["poly_GISAcres"].quantile(0.9),
    caption="Acres Burned",
)

In [ ]:
# Create a color gradient for fire cost
colormap_cost = branca.colormap.LinearColormap(
    colors=[
        "green",
        "yellow",
        "orange",
        "red",
    ],  # Color progression from low to high cost
    vmin=wf_perims["attr_EstimatedCostToDate"].quantile(0),  # Minimum value
    vmax=wf_perims["attr_EstimatedCostToDate"].quantile(0.9),  # 90th percentile
    caption="Estimated Cost",
)

## 5. Map with Color Coded Fire Sizes and Popups

In [ ]:
# Create a base map
map2 = folium.Map(location=[34.406, -110.110], tiles="CartoDB Positron", zoom_start=5)

In [ ]:
# Configure popup information to display when clicking on a fire
popup = folium.GeoJsonPopup(
    fields=[
        "poly_IRWINID",
        "attr_IncidentName",
        "poly_GISAcres",
        "attr_EstimatedCostToDate",
    ],  # Fields to show
    aliases=["IRWINID","Incident Name", "Acres", "Cost"],  # Field names on popup
)

In [ ]:
# Add wildfire perimeters with color based on size
folium.GeoJson(
    wf_perims,
    style_function=lambda x: {
        "fillColor": colormap_size(x["properties"]["poly_GISAcres"]),  # Color based on acres
        "color": colormap_size(x["properties"]["poly_GISAcres"]),  # Border color
    },
    popup=popup,  # Add the popup configuration
).add_to(map2)

In [ ]:
# Add the color legend to the map
colormap_size.add_to(map2)

# Display the map
map2

## 6. Filter Data by State

In [ ]:
# Load US states boundary data
state_data = gpd.read_file(
    "https://raw.githubusercontent.com/python-visualization/folium-example-data/main/us_states.json"
)

# Select California, Texas, and Alaska
target_states = state_data[state_data["id"].isin(["CA", "TX", "AK"])]

# Filter wildfires that intersect with the target states
filtered_wf_perims = wf_perims[wf_perims.geometry.intersects(target_states.union_all())]

# Create a base map centered on Alaska
map3 = folium.Map(location=[66, -150], tiles="CartoDB Positron", zoom_start=4)

# Configure popup information
popup = folium.GeoJsonPopup(
    fields=["attr_IncidentName", "poly_GISAcres", "attr_EstimatedCostToDate"],
    aliases=["Incident Name", "Acres", "Cost"],
    localize=True,
    labels=True,
    style="background-color: blue;",
)

# Add the filtered wildfire perimeters to the map
folium.GeoJson(
    filtered_wf_perims,
    style_function=lambda x: {
        "fillColor": colormap_size(x["properties"]["poly_GISAcres"]),  # Color based on acres
        "color": colormap_size(x["properties"]["poly_GISAcres"]),
    },
    popup=popup,
).add_to(map3)

# Display the map
map3

## 7. Map of Fire Counts by State

In [ ]:
# Create a base map
map4 = folium.Map(location=[34, -110], tiles="CartoDB Positron", zoom_start=4)

In [ ]:
# Load US states boundary data (if not already loaded)
if "state_data" not in locals():
    state_data = gpd.read_file(
        "https://raw.githubusercontent.com/python-visualization/folium-example-data/main/us_states.json"
    )

In [ ]:
# Calculate the count of fires in each state
state_data["fire_count"] = state_data.geometry.apply(
    lambda state_geom: wf_perims.geometry.intersects(state_geom).sum()
)

In [ ]:
# Create a color gradient for fire counts
fire_count_colormap = branca.colormap.LinearColormap(
    colors=["green", "yellow", "orange", "red"],
    vmin=state_data["fire_count"].min(),
    vmax=state_data["fire_count"].max(),
    caption="Fire Count",
)

In [ ]:
# Add state boundaries with colors based on fire count
folium.GeoJson(
    state_data,
    style_function=lambda x: {
        "fillColor": fire_count_colormap(x["properties"]["fire_count"]),
        "color": "black",
        "weight": 1,
        "dashArray": "2,5",
    },
    popup=folium.GeoJsonPopup(fields=["fire_count"]),
).add_to(map4)

In [ ]:
# Configure popup information for fire perimeters
popup = folium.GeoJsonPopup(
    fields=["attr_IncidentName", "poly_GISAcres", "attr_EstimatedCostToDate"],
    aliases=["Incident Name", "Acres", "Cost"],
    localize=True,
    labels=True,
    style="background-color: blue;",
)

In [ ]:
# Add wildfire perimeters with color based on size
folium.GeoJson(
    wf_perims,
    style_function=lambda x: {
        "fillColor": colormap_size(x["properties"]["poly_GISAcres"]),
        "color": colormap_size(x["properties"]["poly_GISAcres"]),
    },
    popup=popup,
).add_to(map4)

In [ ]:
# Add the fire count legend to the map
fire_count_colormap.add_to(map4)

# Display the map
map4

## Summary

In this notebook, we've demonstrated how to:

1. Load and preprocess geospatial wildfire data using GeoPandas
2. Create interactive maps with Folium
3. Add popups to display detailed information
4. Use color gradients to visualize fire sizes
5. Filter data by geographic regions
6. Create choropleth maps to show fire counts by state

These techniques can be applied to many other geospatial datasets and analysis scenarios.